## The following colab notebook, is an implementation to use the stablecode-instruct-alpha-3b by [StabilityAI](https://huggingface.co/stabilityai) as an Flask(ngrok) based api

### This notebook was created as a part of StableCode 24-hours Hackathon at [LabLab.ai](https://lablab.ai)

In [1]:
%%capture
!pip install transformers
!pip install flask
!pip install flask_ngrok

## Get your Hugging Face access token from [Hugging Face Tokens page](https://huggingface.co/settings/tokens)

In [2]:
access_token = "YOUR_TOKEN_HERE"

## Download the model weights from HuggingFace

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("stabilityai/stablecode-instruct-alpha-3b",
                                          use_auth_token=access_token
                                          )

model = AutoModelForCausalLM.from_pretrained(
    "stabilityai/stablecode-instruct-alpha-3b",
    trust_remote_code=True,
    use_auth_token=access_token,
    torch_dtype="auto"
    )

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:631: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:460: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:373: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


In [4]:
%%capture
model.cuda()

### Test run before actually setting up the API like system





In [5]:
inputs = tokenizer("###Instruction\nGenerate a python function to find number of CPU cores###Response\n", return_tensors="pt",return_token_type_ids=False).to("cuda")
tokens = model.generate(
  **inputs,
  max_new_tokens=2048,
  temperature=1,
  do_sample=True,
)
print(tokenizer.decode(tokens[0], skip_special_tokens=True))

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1411: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


###Instruction
Generate a python function to find number of CPU cores###Response

def countCPUs():
    return(os.cpu_count())


## Main model generation function

In [6]:
def generate_text(input):
  prompt=input
  inputs = tokenizer(f"###Instruction\n{prompt}###Response\n", return_tensors="pt",return_token_type_ids=False).to("cuda")
  tokens = model.generate(
  **inputs,
  max_new_tokens=2048,
  temperature=1,
  do_sample=True,
  )
  return (tokenizer.decode(tokens[0], skip_special_tokens=True))

#### The following cell would return a few links, you have to choose a link which looks like http://e3f6-34-87-122-30.ngrok.io add '/process_string' to the end to make it look like http://e3f6-34-87-122-30.ngrok.io/process_string and make the api calls using curl, postman or your code.

In [7]:
from flask import Flask, request, jsonify
import requests

app = Flask(__name__)

@app.route('/process_string', methods=['POST'])
def process_string_api():
    data = request.get_json()

    if 'input_string' not in data:
        return jsonify({'error': 'Input string not provided'}), 400

    input_string = data['input_string']
    result = generate_text(input_string)
    return jsonify({'result': result}), 200

if __name__ == '__main__':
    from flask_ngrok import run_with_ngrok
    run_with_ngrok(app)
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://16f2-34-126-134-108.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
INFO:werkzeug:127.0.0.1 - - [25/Aug/2023 15:08:42] "POST /process_string HTTP/1.1" 200 -
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
INFO:werkzeug:127.0.0.1 - - [25/Aug/2023 15:12:12] "POST /process_string HTTP/1.1" 200 -
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
INFO:werkzeug:127.0.0.1 - - [25/Aug/2023 15:19:25] "POST /process_string HTTP/1.1" 200 -
